In [165]:
# Packages

import math
import pandas as pd
import numpy as np
import re

from datetime import datetime
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics
from sklearn import cross_validation
from sklearn import linear_model
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import csv

In [2]:
# Parameters

train='C:/Users/m01i795/iPython Notebooks/Kaggle_springleaf/Input/train.csv'
test='C:/Users/m01i795/iPython Notebooks/Kaggle_springleaf/Input/test.csv'

In [3]:
# Now, reading in the first 1500 rows to keep memory use low
features_probe = pd.read_csv(train, nrows=200, low_memory=False)

In [4]:
features_probe.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
ID,2,4,5,7,8,14,16,20,21,22,23,24,25,26,28
VAR_0001,H,H,H,H,R,R,H,R,R,R,R,H,H,R,Q
VAR_0002,224,7,116,240,72,4,60,13,17,24,61,140,258,43,36
VAR_0003,0,53,3,300,261,4,132,75,16,72,12,0,14,126,99
VAR_0004,4300,4448,3464,3200,2000,4422,40000,3600,2296,450,1186,1962,3500,4000,1500
VAR_0005,C,B,C,C,N,C,C,B,N,N,C,C,B,B,B
VAR_0006,0,1,0,0,0,0,1,0,1,0,1,0,6,0,0
VAR_0007,0,0,0,0,0,0,1,0,1,0,1,0,2,0,0
VAR_0008,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
VAR_0009,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
# Fastest way to get dtypes of dataframe:
features_probe.get_dtype_counts()

bool         13
float64     290
int64      1594
object       37
dtype: int64

In [6]:
features_probe['VAR_0008'].value_counts()

False    200
dtype: int64

In [106]:
# OK, now reading in a small, but substantial, file, but this time I leave out the low_memory=False (I'm specifying the columns myself)
features_small = pd.read_csv(train, nrows=10000, dtype=features_probe.dtypes, na_values=[''])

In [107]:
features_small['VAR_0207'].value_counts(dropna=False)

NaN    10000
dtype: int64

In [108]:
features_small.get_dtype_counts()

object    1934
dtype: int64

In [109]:
# The above types are a pity in terms of output: I hoped to have casted the variables to the proper types :(

In [110]:
# Cleaning the ID and target columns respectively
labels_small=features_small['target']
features_small.drop(['ID', 'target'], axis=1, inplace=True)

In [111]:
# Removing the missing values with an easier to handle value
features_small.dropna(axis=1, how='all', inplace=True)
features_small.dropna(axis=0, how='all', inplace=True)
features_small.fillna(-9999, inplace=True)
features_small.shape

(10000, 1929)

In [112]:
# Getting a visual inspection of the data
for ii in features_small.columns:
    print "Checking: " + ii
    print "Top 10 values:"
    print "="*40
    print features_small[ii].value_counts(dropna=False)[:10]
    print
    print "First 10 values: "
    print "-+"*20
    for jj in range(10):
        print features_small[ii][jj]
    print

Checking: VAR_0001
Top 10 values:
R    5758
H    4202
Q      40
dtype: int64

First 10 values: 
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
H
H
H
H
R
R
H
R
R
R

Checking: VAR_0002
Top 10 values:
12    332
24    317
36    259
60    209
13    197
6     179
2     153
3     152
48    139
1     136
dtype: int64

First 10 values: 
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
224
7
116
240
72
4
60
13
17
24

Checking: VAR_0003
Top 10 values:
0     1203
24     234
60     224
12     214
36     162
6      134
72     131
48     124
1      117
84     116
dtype: int64

First 10 values: 
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
0
53
3
300
261
4
132
75
16
72

Checking: VAR_0004
Top 10 values:
2000    218
3000    211
2500    200
0       198
4000    155
2400    151
1500    137
1800    135
1600    127
3500    120
dtype: int64

First 10 values: 
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
4300
4448
3464
3200
2000
4422
40000
3600
2296
450

Checking: VAR_0005
Top 10 values:
B    5021
C    3676
N    1120
S     183
dty

In [113]:
def timetodaysago (timeobs):
    # input: timeobs is of format 23NOV10:00:00:00
    # output: days ago since September 1st, 2015
    
    try:
        d0= datetime.strptime(timeobs[:7], '%d%b%y')
        d1= datetime.strptime('01SEP15', '%d%b%y')
        delta = d1 - d0
        return delta.days
    except:
        return timeobs

# Test by removing # in line below
# timetodaysago('04AUG12:00:00:00') 

In [114]:
def areallvalues(series):
    try:
        series = series.astype('float64')
    except:
        return False        
    return True

In [115]:
# Processing the columns of the dataframe:

# Table above suggests that there are five situations for categorical columns:
# 1) Drop columns if only 1 value is there (useless)
# 2) Transform timestamps to a continuous variable
# 3) One-hot encoding for columns with 1 < cardinality < 60
# 4) Cardinality > 60 and values -> cast them to float64
# 5) Cardinality > 60 and 'names' >> TODO binning (for now, dummies for largest variables)

for ii in features_small.columns:
    print
    print 'Checking: ' + str(ii)
    top_keys = pd.Series(list(features_small[ii].value_counts().keys()))
    top_values = pd.Series(list(features_small[ii].value_counts()))    
    print
    print top_keys[:10]
    print   
    to_check_series = top_keys[top_keys<>-9999]
    to_check = list(to_check_series)[0] # This ugly trick is necessary as the index doesn't start at 0 for a selected DF :(
    print 'To check: ' + str(to_check)
    raw_cardinality = len(top_keys.value_counts()) # Note that although a variable might be only False, NA then also contains info
    print 'Raw Cardinality: ' + str(raw_cardinality)

    
    if raw_cardinality <= 1: # So there is max a single value in all columns (situation 1) --> drop!
        print 'Only one or zero distinct values --> drop'
        features_small.drop(ii, axis=1, inplace=True)
    else: # Checking whether we have situation 2 or 3/4/5 now...

        
        print 'Checking for datetimeformat: ' + str(to_check) # Note that raw cardinality > 1, so there has to be a non-null value

        p = re.compile(r'\w{7}:\d{2}:\d{2}:\d{2}') # Regular expression for datetime format used

        if p.match(str(to_check)): # Situation 2: the match returns something > it's of a datetime format 
            print 'Timestamp format --> mapping to timetodaysago'
            features_small[ii]=features_small[ii].map(timetodaysago)
            features_small[ii] = features_small[ii].astype('float64')
            
        else: # Checking whether we have situation 3 or 4/5 now...           
            if raw_cardinality < 60: # This would be situation 3 --> Let's one hot encode this!
                print 'Making ' + str(raw_cardinality) + ' dummies'
                toadd = pd.get_dummies(features_small[ii], prefix=ii, sparse=True) # Generate dummies
                features_small = pd.concat([features_small, toadd], axis=1) # Add the dummies to the DataFrame
                features_small.drop(ii, axis=1, inplace=True) # Dropping the original column, as it has been one hot encoded now.
            
            else: # Checking whether we have situation 4 or 5 now...                
                if areallvalues(features_small[ii]): # Situation 4 is easiest...
                    print 'Raw cardinality >60 detected, and types can be casted to float64. So doing that now...'
                    features_small[ii]=features_small[ii].astype('float64')
                else: # So, situation 5 with large cardinality and few observations (TODO: binning!!)
                    top_keys = top_keys[top_values>30] # So, observations with n>30 get a seperate dummy                   
                    cardinality_corrected=len(top_keys)
                    print 'Cardinality corrected for at least 30 observations: ' + str(cardinality_corrected)
                    if cardinality_corrected >= 1:
                        print 'Making ' + str(cardinality_corrected + 1) + ' dummies'
                        features_small[ii]=features_small[ii].map(lambda x: x if (x in list(top_keys)) else ('_other')) # And maps the values to this top (or var_other)
                        toadd = pd.get_dummies(features_small[ii], prefix=ii, sparse=True) # Generate dummies (for variables with n > 30)
                        features_small = pd.concat([features_small, toadd], axis=1) # Add the dummies to the DataFrame
                    features_small.drop(ii, axis=1, inplace=True) # Dropping the original column, 
                                                                  # as it has been one hot encoded now (or it's too sparse)



Checking: VAR_0001

0    R
1    H
2    Q
dtype: object

To check: R
Raw Cardinality: 3
Checking for datetimeformat: R
Making 3 dummies

Checking: VAR_0002

0    12
1    24
2    36
3    60
4    13
5     6
6     2
7     3
8    48
9     1
dtype: object

To check: 12
Raw Cardinality: 556
Checking for datetimeformat: 12
Raw cardinality >60 detected, and types can be casted to float64. So doing that now...

Checking: VAR_0003

0     0
1    24
2    60
3    12
4    36
5     6
6    72
7    48
8     1
9    84
dtype: object

To check: 0
Raw Cardinality: 445
Checking for datetimeformat: 0
Raw cardinality >60 detected, and types can be casted to float64. So doing that now...

Checking: VAR_0004

0    2000
1    3000
2    2500
3       0
4    4000
5    2400
6    1500
7    1800
8    1600
9    3500
dtype: object

To check: 2000
Raw Cardinality: 3000
Checking for datetimeformat: 2000
Raw cardinality >60 detected, and types can be casted to float64. So doing that now...

Checking: VAR_0005

0    B
1    C

In [ ]:
# Preprocessing done... some tests follow

In [116]:
# Checking the number of rows and columns (columns grow large due to one-hot encoding)
features_small.shape

(10000, 18814)

In [117]:
# Checking data types (should be all float64 now)
features_small.get_dtype_counts()

float64    18814
dtype: int64

In [119]:
# Pushing to np.arrays for proper handling by Sklearn
X = np.asarray(features_small)
Y = np.asarray(labels_small)

In [120]:
# Check whether there are still nan's in the np.array X
print np.isnan(np.sum(X))

False


In [ ]:
# Starting the ML part

In [164]:
# Splitting the data to train and test set
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.2, random_state=42)

In [123]:
clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [124]:
clf.fit(X_train, y_train)

In [125]:
clf.score(X_test, y_test)

0.79400000000000004

In [137]:
scores = cross_validation.cross_val_score(clf, X, Y, cv=5)

In [138]:
scores

array([ 0.78210895,  0.79      ,  0.793     ,  0.7895    ,  0.7913957 ])

In [163]:
y_predict=clf.predict(X_test)

In [155]:
y_test=y_test.astype(int)
y_predict=y_predict.astype(int)

In [161]:
sum(y_predict * y_test)

46

In [158]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict, pos_label=2)

In [162]:
metrics.roc_auc_score(y_test, y_predict)

0.54457117464079396

In [166]:
clf2 = ExtraTreesClassifier (n_estimators=100, n_jobs=-1, verbose=2)

In [167]:
clf2.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done   1 out of 100 | elapsed:    1.1s remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  51 out of 100 | elapsed:   17.1s remaining:   16.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   33.2s finished


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=2, warm_start=False)

In [168]:
y_predict2 = clf2.predict(X_test)

[Parallel(n_jobs=4)]: Done   1 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   8 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   9 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  27 out of  51 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  28 out of  53 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  29 out of  55 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  51 out of  99 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [170]:
y_predict2

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

In [ ]:
# Loading the full dataset now in an efficient way
# Need to change to skiprows=1 for final version, this is just to speed things up a bit
features_train = pd.read_csv(train, dtype=features_small.dtypes, names=coltxt, skiprows=100000)

In [ ]:
labels_train = features_train['target']

In [ ]:
# How to find a value easily in a Pandas series?

for col in features_train.columns:
    if len(features_train[features_train[col]=='IAPS'])>0:
        print "IAPS found in " + col

In [ ]:
features_small.select_dtypes(include=[object]).columns

In [ ]:
for ii in cardinality.columns:
    print ii + ": " + str(len(features_train[ii].value_counts()))

In [ ]:
%matplotlib inline
features_small['VAR_0004'].value_counts().plot(kind='bar')

In [ ]:
toadd = pd.get_dummies(features_train['VAR_0001'], sparse=False)

In [ ]:
features_train = pd.concat([features_train, toadd], axis=1)

In [ ]:
features_train.head(5)

In [ ]:
clf = tree.DecisionTreeClassifier()

In [ ]:
clf = clf.fit(features_train, labels_train)

In [ ]:
# clf = linear_model.SGDClassifier()

In [ ]:
# clf.fit(features_train, labels_train)

In [ ]:
pd.read_csv?

In [ ]:
# Alternative, somewhat longer :
features_probe.dtypes.value_counts()

In [ ]:
# Alternative, even longer :)
# filter(lambda s: s == np.dtype('O'), features_small.dtypes.values)

{d: sum(1 for _ in filter(lambda s: s == d, features_probe.dtypes.values)) for d in np.unique(features_probe.dtypes.values)}

In [ ]:
col_types = features_small.columns
categorical_cols = [ii for ii in features_small.select_dtypes(include=[object]).columns]
bool_cols = [ii for ii in features_small.select_dtypes(include=[bool]).columns]            
value_cols = [ii for ii in features_small.select_dtypes(include=['int64', 'float64']).columns]

In [ ]:
row = {'var1': 200, 'var2': 'a', 'var3': True}

x = []
for key in row:
    print key
    value = row[key]
    # one-hot encode everything with hash trick
    index = abs(hash(key + '_' + value)) % D
    x.append(index)

print x

In [ ]:
# Processing the columns of the dataframe:

# Table above suggests that there are three situations for categorical columns:
# 1) Drop columns if only 1 value is there (useless)
# 2) Transform timestamps to a continuous variable
# 3) One-hot encoding for columns with 1 < cardinality < 60
# 4) Cast the column to float64 if cardinality > 60

for ii in features_small.columns
    raw_cardinality = len(features_small[ii].value_counts())
    print
    print 'Checking: ' + str(ii)
    print 'Raw Cardinality: ' + str(raw_cardinality)
    
    if raw_cardinality <= 1: # So there is max a single value in all columns (situation 1) --> drop!
        print 'Only one or zero distinct values --> drop'
        features_small.drop(ii, axis=1, inplace=True)
    else: # Checking whether we have situation 2 or 3/4 now...
        top_keys = pd.Series(list(features_small[ii].value_counts().keys()))
        top_values = pd.Series(list(features_small[ii].value_counts()))    

        to_check = top_keys[~pd.isnull(top_keys)][0]        
        print 'Checking for datetimeformat: ' + str(to_check) # Note that raw cardinality > 1, so there has to be a non-null value

        p = re.compile(r'\w{7}:\d{2}:\d{2}:\d{2}') # Regular expression for datetime format used
        # Situation 2:
        if p.match(str(to_check)): # Situation 2: the match returns something > it's of a datetime format 
            print 'Timestamp format --> mapping to timetodaysago'
            features_small[ii]=features_small[ii].map(timetodaysago)
            features_small[ii].fillna(-9999)
            features_small[ii] = features_small[ii].astype('float64')
            
        else:            
            
            
            top_keys = top_keys[top_values>30] # So, keep only observations with n>30
            cardinality_corrected=len(top_keys)
            print 'Cardinality corrected: ' + str(cardinality_corrected)
            print top_keys
            
            if cardinality_corrected > 1: # So, this means less than 50 unique values > making dummies
            
                if cardinality_corrected < raw_cardinality: # if they are equal, don't add 'other' column, just make dummies
                    print 'Making ' + str(cardinality_corrected + 1) + ' dummies'
                    features_small[ii]=features_small[ii].map(lambda x: x if (x in list(top_keys)) else ('_other')) # And maps the values to this top (or var_other)
                else:
                    print 'Making ' + str(cardinality_corrected) + ' dummies (no additional _other needed)'
                toadd = pd.get_dummies(features_small[ii], prefix=ii, sparse=False) # Generate dummies (for variables with n > 30)
                features_small = pd.concat([features_small, toadd], axis=1) # Add the dummies to the DataFrame
                features_small.drop(ii, axis=1, inplace=True) # Dropping the original column, as it has been one hot encoded now.
            else:
                print 'Treating this column as boolean: 1 < raw cardinality < 50 and corrected cardinality <= 1'
                top_keys = pd.Series(list(features_small[ii].value_counts().keys()))
                top_keys = list(top_keys[~pd.isnull(top_keys)])
                print 'List of values as 1: ' + str(top_keys)
                features_small[ii]=features_small[ii].map(lambda x: np.nan if (pd.isnull(x)) else 1 if (x in top_keys) else 0)

In [ ]:
# Processing the columns with bool values
# None left :(
for ii in bool_cols:
    top_values = pd.Series(list(features_small[ii].value_counts())) 
    top_keys = pd.Series(list(features_small[ii].value_counts().keys()))
    raw_cardinality = len(top_values)
    
    top_keys = top_keys[top_values>30] # So, keep only observations with n>30
    raw_cardinality = len(features_small[ii].value_counts())
    print
    print 'Checking: ' + str(ii)
    print 'Raw Cardinality: ' + str(raw_cardinality)
    
    if raw_cardinality <= 1: # So there is max a single value in all columns (situation 1) --> drop!
        print 'Only one or zero distinct value --> drop'
        features_small.drop(ii, axis=1, inplace=True)
    else:
        print 'Proper column, so replacing True with 1, False with 0 and keeping nan'
        features_small[ii]=features_small[ii].map(lambda x: np.nan if (pd.isnull(x)) else 1 if (x) else 0)

In [ ]:
# Processing the columns with integer or float values
# None left :(
for ii in value_cols:
    raw_cardinality = len(features_small[ii].value_counts())
    print
    print 'Checking: ' + str(ii)
    print 'Raw Cardinality: ' + str(raw_cardinality)
    
    if raw_cardinality <= 1: # So there is max a single value in all columns (situation 1) --> drop!
        print 'Only one or zero distinct value --> drop'
        features_small.drop(ii, axis=1, inplace=True)
        
    elif raw_cardinality < 50: # This means that it's more categorical than a true number
        print 'Making ' + str(raw_cardinality) + ' dummies (no additional _other needed)'
        toadd = pd.get_dummies(features_small[ii], prefix=ii, sparse=False) # Generate dummies (for variables with n > 30)
        features_small = pd.concat([features_small, toadd], axis=1) # Add the dummies to the DataFrame
        features_small.drop(ii, axis=1, inplace=True) # Dropping the original column, as it has been one hot encoded now.
    else:
        print 'Keeping this column as it is currently'
